# Ejemplo ETL con Polars: Dataset de Taxis de Nueva York

En esta sección, implementaremos un ejemplo completo de ETL (Extracción, Transformación y Carga) utilizando Polars para procesar el dataset de taxis de Nueva York. Este ejemplo demostrará las ventajas de Polars sobre Pandas en términos de rendimiento y funcionalidades.

Nuestro ETL incluirá:
1. Extracción de datos desde archivos Parquet
2. Transformación y limpieza de datos con Polars
3. Validación de datos con Pydantic
4. Carga de datos en una base de datos SQLite utilizando SQLAlchemy
5. Implementación de DAGs (Directed Acyclic Graphs)
6. Configuración de logging para seguimiento del proceso

Comencemos explorando la estructura del proyecto y los componentes principales.

## Estructura del Proyecto

Nuestro proyecto ETL está organizado de la siguiente manera:

```
notebook_polars_pyspark/
├── data/
│   └── yellow_tripdata.parquet  # Dataset de taxis de Nueva York
├── etl_example/
│   ├── __init__.py
│   ├── etl_config.py      # Configuración del ETL
│   ├── models.py          # Modelos Pydantic para validación
│   ├── database.py        # Configuración de SQLAlchemy
│   ├── logger.py          # Configuración de logging
│   ├── etl_dag.py         # Implementación de DAGs
│   ├── output/            # Directorio para la base de datos
│   └── logs/              # Directorio para logs
└── notebooks/
```

Vamos a examinar cada componente del ETL en detalle.

## 1. Configuración del ETL (etl_config.py)

El archivo `etl_config.py` contiene la configuración básica para nuestro ETL, incluyendo rutas de archivos, configuración de la base de datos y parámetros de logging.

In [1]:
# Mostrar el contenido del archivo etl_config.py
from pathlib import Path

contenido = Path("etl_example/etl_config.py").read_text(encoding="utf-8")
print(contenido)

"""
Configuración para el ETL de taxis de Nueva York
"""
import os
from pathlib import Path

# Rutas de archivos
BASE_DIR = Path(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
DATA_DIR = BASE_DIR / "data" / "processed"
OUTPUT_DIR = BASE_DIR / "etl_example" / "output"
LOG_DIR = BASE_DIR / "etl_example" / "logs"

# Asegurar que los directorios existan
OUTPUT_DIR.mkdir(exist_ok=True)
LOG_DIR.mkdir(exist_ok=True)

# Configuración de la base de datos
DB_PATH = OUTPUT_DIR / "nyc_taxi.db"
DB_URI = f"sqlite:///{DB_PATH}"

# Configuración de logging
LOG_FILE = LOG_DIR / "etl_process.log"
LOG_LEVEL = "INFO"
LOG_FORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"

# Configuración del dataset
TAXI_DATA_FILE = DATA_DIR / "yellow_tripdata.parquet"
TAXI_DATA_FILE_SMALL = DATA_DIR / "yellow_tripdata_small.parquet"

# Configuración de procesamiento
BATCH_SIZE = 100000  # Número de filas a procesar en cada lote



## 2. Modelos de Datos con Pydantic (models.py)

Utilizamos Pydantic para definir modelos de datos con validación estricta de tipos. Esto nos permite asegurar que los datos cumplen con nuestras expectativas antes de cargarlos en la base de datos.

In [2]:
# Mostrar el contenido del archivo models.py
contenido = Path("etl_example/models.py").read_text(encoding="utf-8")
print(contenido)

"""
Modelos de datos para el ETL de taxis de Nueva York usando Pydantic
"""
from datetime import datetime
from typing import Optional
from pydantic import BaseModel, Field, validator

class TaxiTrip(BaseModel):
    """Modelo Pydantic para validar los datos de viajes de taxi"""
    
    # Campos de identificación
    vendor_id: int = Field(..., description="ID del proveedor del taxi")
    
    # Campos de tiempo
    pickup_datetime: datetime = Field(..., description="Fecha y hora de recogida")
    dropoff_datetime: datetime = Field(..., description="Fecha y hora de entrega")
    
    # Campos de ubicación
    pickup_location_id: int = Field(..., description="ID de la ubicación de recogida")
    dropoff_location_id: int = Field(..., description="ID de la ubicación de entrega")
    
    # Campos de pasajeros
    passenger_count: Optional[float] = Field(None, description="Número de pasajeros")
    
    # Campos de distancia
    trip_distance: float = Field(..., description="Distancia del v

### Ventajas de Pydantic para Validación de Datos

Pydantic ofrece varias ventajas para la validación de datos en flujos ETL:

1. **Validación de tipos en tiempo de ejecución**: Pydantic valida automáticamente los tipos de datos y convierte valores cuando es posible.
2. **Validadores personalizados**: Podemos definir funciones de validación personalizadas para reglas de negocio específicas.
3. **Documentación integrada**: Los modelos Pydantic son autodocumentados con descripciones de campos.
4. **Integración con FastAPI y otras bibliotecas**: Pydantic se integra bien con el ecosistema de Python.
5. **Manejo de errores detallado**: Proporciona mensajes de error claros cuando la validación falla.

## 3. Configuración de la Base de Datos con SQLAlchemy (database.py)

Utilizamos SQLAlchemy para definir el esquema de la base de datos y gestionar las conexiones. SQLAlchemy nos permite trabajar con bases de datos de manera orientada a objetos.

In [3]:
# Mostrar el contenido del archivo database.py
contenido = Path("etl_example/database.py").read_text(encoding="utf-8")
print(contenido)

"""
Configuración de la base de datos SQLite para el ETL de taxis de Nueva York
"""
from sqlalchemy import Column, Integer, Float, String, DateTime, ForeignKey, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

from etl_example.etl_config import DB_URI

# Crear la base para los modelos SQLAlchemy
Base = declarative_base()

class TaxiLocation(Base):
    """Modelo SQLAlchemy para la tabla de ubicaciones"""
    __tablename__ = 'locations'
    
    location_id = Column(Integer, primary_key=True)
    borough = Column(String, nullable=False)
    zone = Column(String, nullable=False)
    service_zone = Column(String, nullable=False)
    
    # Relación con los viajes (pickup)
    pickup_trips = relationship("TaxiTripRecord", foreign_keys="TaxiTripRecord.pickup_location_id", back_populates="pickup_location")
    # Relación con los viajes (dropoff)
    dropoff_trips = relationship("TaxiTripRecord", foreign_keys="TaxiTrip

### Ventajas de SQLAlchemy para ETL

SQLAlchemy ofrece varias ventajas para los procesos ETL:

1. **Abstracción de la base de datos**: Podemos cambiar el motor de base de datos sin modificar el código.
2. **Mapeo objeto-relacional (ORM)**: Trabajamos con objetos Python en lugar de SQL directo.
3. **Gestión de sesiones**: Manejo eficiente de transacciones y conexiones.
4. **Migraciones de esquema**: Facilita la evolución del esquema de la base de datos.
5. **Validación a nivel de base de datos**: Complementa la validación de Pydantic con restricciones a nivel de base de datos.

## 4. Configuración de Logging (logger.py)

El sistema de logging nos permite seguir el progreso del ETL y diagnosticar problemas.

In [4]:
# Mostrar el contenido del archivo logger.py
contenido = Path("etl_example/logger.py").read_text(encoding="utf-8")
print(contenido)

"""
Configuración del sistema de logging para el ETL de taxis de Nueva York
"""
import logging
import sys
from pathlib import Path

from etl_example.etl_config import LOG_FILE, LOG_LEVEL, LOG_FORMAT

def setup_logger(name, log_file=LOG_FILE, level=LOG_LEVEL):
    """Configura y devuelve un logger con el nombre especificado"""
    
    # Crear el directorio de logs si no existe
    log_dir = Path(log_file).parent
    log_dir.mkdir(exist_ok=True)
    
    # Configurar el logger
    logger = logging.getLogger(name)
    
    # Establecer el nivel de logging
    level_obj = getattr(logging, level)
    logger.setLevel(level_obj)
    
    # Crear un manejador para archivo
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(level_obj)
    
    # Crear un manejador para consola
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(level_obj)
    
    # Crear el formato
    formatter = logging.Formatter(LOG_FORMAT)
    file_handler.setFormatt

## 5. Implementación de DAGs (etl_dag.py)

Para implementar DAGs (Directed Acyclic Graphs) que definen el flujo de trabajo del ETL.

In [5]:
# Mostrar el contenido del archivo etl_dag.py
contenido = Path("etl_example/etl_dag.py").read_text(encoding="utf-8")
print(contenido)

"""
Implementación de ETL para taxis de Nueva York usando sistema de DAGs personalizado
"""
from typing import List, Dict, Any
import time

import polars as pl
from sqlalchemy.orm import Session

from etl_example.etl_config import TAXI_DATA_FILE_SMALL, BATCH_SIZE
from etl_example.logger import setup_logger
from etl_example.models import TaxiTrip
from etl_example.database import init_db, get_session, TaxiTripRecord, TaxiLocation
from utils import DAG

# Configurar el logger
logger = setup_logger("nyc_taxi_etl")

# Crear el DAG
taxi_dag = DAG(
    dag_id="nyc_taxi_etl",
    description="ETL para procesar datos de taxis de Nueva York"
)

def extract_taxi_data(file_path: str = str(TAXI_DATA_FILE_SMALL)) -> pl.DataFrame:
    """Extrae los datos del archivo parquet utilizando Polars"""
    logger.info(f"Extrayendo datos del archivo: {file_path}")

    try:
        # Usar Polars para leer el archivo parquet
        df = pl.read_parquet(file_path)

        # Renombrar columnas para que coincid

## Ejecutando el ETL

Ahora vamos a ejecutar nuestro ETL y analizar su rendimiento. Primero, importamos los módulos necesarios y configuramos el entorno.

In [6]:
import sys
import time
from pathlib import Path

# Añadir el directorio raíz al path para poder importar los módulos
sys.path.append(str(Path.cwd().parent))

# Importar los módulos del ETL
from etl_example.etl_dag import nyc_taxi_etl_flow
from etl_example.etl_config import DB_PATH, OUTPUT_DIR

# Asegurar que el directorio de salida existe
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# Eliminar la base de datos si existe para empezar desde cero
if DB_PATH.exists():
    DB_PATH.unlink()

print(f"Configuración completada. La base de datos se creará en: {DB_PATH}")

Configuración completada. La base de datos se creará en: /Users/isaromobru/Desktop/DS102024_/4-DataEngineer/PolarsPySpark/etl_example/output/nyc_taxi.db


Ahora ejecutamos el flujo ETL y medimos el tiempo que tarda en completarse.

In [7]:
# Ejecutar el flujo ETL y medir el tiempo
start_time = time.time()

# Ejecutar el flujo
nyc_taxi_etl_flow()

end_time = time.time()
execution_time = end_time - start_time

print(f"\nETL completado en {execution_time:.2f} segundos")

2025-04-09 20:40:40,804 - nyc_taxi_etl - INFO - Iniciando flujo ETL para datos de taxis de Nueva York


2025-04-09 20:40:40,804 [INFO] Iniciando flujo ETL para datos de taxis de Nueva York


2025-04-09 20:40:40,807 - nyc_taxi_etl - INFO - Extrayendo datos del archivo: /Users/isaromobru/Desktop/DS102024_/4-DataEngineer/PolarsPySpark/data/processed/yellow_tripdata_small.parquet


2025-04-09 20:40:40,807 [INFO] Extrayendo datos del archivo: /Users/isaromobru/Desktop/DS102024_/4-DataEngineer/PolarsPySpark/data/processed/yellow_tripdata_small.parquet


2025-04-09 20:40:41,366 - nyc_taxi_etl - INFO - Datos extraídos exitosamente. Filas: 2964624, Columnas: 19


...

2025-04-09 20:53:52,154 - nyc_taxi_etl - INFO - Lote procesado: 2300001 a 2400000 de 2869996


2025-04-09 20:53:52,154 [INFO] Lote procesado: 2300001 a 2400000 de 2869996


2025-04-09 20:54:10,225 - nyc_taxi_etl - INFO - Lote procesado: 2400001 a 2500000 de 2869996


2025-04-09 20:54:10,225 [INFO] Lote procesado: 2400001 a 2500000 de 2869996


KeyboardInterrupt: 

## Verificando los Resultados

Vamos a verificar que los datos se hayan cargado correctamente en la base de datos SQLite.

In [4]:
import sqlite3
import polars as pl

# Conectar a la base de datos
conn = sqlite3.connect(str(DB_PATH))

# Consultar usando read_database
trip_count = pl.read_database(query="SELECT COUNT(*) FROM taxi_trips", connection=conn).item(0, 0)
location_count = pl.read_database(query="SELECT COUNT(*) FROM locations", connection=conn).item(0, 0)

print(f"Número de viajes en la base de datos: {trip_count}")
print(f"Número de ubicaciones en la base de datos: {location_count}")

# Consultar algunos viajes para verificar
sample_trips = pl.read_database(query="SELECT * FROM taxi_trips LIMIT 5", connection=conn)

print("\nMuestra de viajes:")
print(sample_trips)

conn.close()

Número de viajes en la base de datos: 2869996
Número de ubicaciones en la base de datos: 262

Muestra de viajes:
shape: (5, 18)
┌─────┬───────────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ id  ┆ vendor_id ┆ pickup_dat ┆ dropoff_da ┆ … ┆ total_amou ┆ congestion ┆ airport_fe ┆ payment_t │
│ --- ┆ ---       ┆ etime      ┆ tetime     ┆   ┆ nt         ┆ _surcharge ┆ e          ┆ ype       │
│ i64 ┆ i64       ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---       │
│     ┆           ┆ str        ┆ str        ┆   ┆ f64        ┆ f64        ┆ f64        ┆ i64       │
╞═════╪═══════════╪════════════╪════════════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ 1   ┆ 2         ┆ 2024-01-01 ┆ 2024-01-01 ┆ … ┆ 22.7       ┆ 2.5        ┆ 0.0        ┆ 2         │
│     ┆           ┆ 00:57:55.0 ┆ 01:17:43.0 ┆   ┆            ┆            ┆            ┆           │
│     ┆           ┆ 00000      ┆ 00000      ┆   ┆            ┆  

## Comparación de Rendimiento: Polars vs Pandas

Para demostrar las ventajas de rendimiento de Polars sobre Pandas, vamos a implementar una versión simplificada del mismo proceso ETL utilizando Pandas y comparar los tiempos de ejecución.

In [5]:
import pandas as pd
import time
from etl_example.etl_config import TAXI_DATA_FILE

def etl_with_pandas():
    # Extracción
    start_time = time.time()
    print("Extrayendo datos con Pandas...")
    df_pandas = pd.read_parquet(TAXI_DATA_FILE)
    extraction_time = time.time() - start_time
    print(f"Extracción completada en {extraction_time:.2f} segundos")
    
    # Transformación
    start_time = time.time()
    print("Transformando datos con Pandas...")
    
    # Renombrar columnas para consistencia
    column_mapping = {
        "VendorID": "vendor_id",
        "tpep_pickup_datetime": "pickup_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "PULocationID": "pickup_location_id",
        "DOLocationID": "dropoff_location_id"
    }
    df_pandas = df_pandas.rename(columns=column_mapping)
    
    # Filtrar viajes con distancia válida
    df_pandas = df_pandas[df_pandas['trip_distance'] > 0]
    
    # Filtrar viajes con tarifa válida
    df_pandas = df_pandas[df_pandas['fare_amount'] >= 0]
    
    # Calcular la duración del viaje en minutos
    df_pandas['trip_duration_minutes'] = (df_pandas['dropoff_datetime'] - df_pandas['pickup_datetime']).dt.total_seconds() / 60
    
    # Filtrar viajes con duración válida
    df_pandas = df_pandas[df_pandas['trip_duration_minutes'] > 0]
    
    # Calcular la velocidad promedio
    df_pandas['avg_speed_mph'] = df_pandas['trip_distance'] / (df_pandas['trip_duration_minutes'] / 60)
    
    # Filtrar velocidades razonables
    df_pandas = df_pandas[df_pandas['avg_speed_mph'] < 100]
    
    # Manejar valores nulos
    df_pandas['passenger_count'] = df_pandas['passenger_count'].fillna(1)
    df_pandas['congestion_surcharge'] = df_pandas['congestion_surcharge'].fillna(0)
    df_pandas['Airport_fee'] = df_pandas['Airport_fee'].fillna(0)
    
    transformation_time = time.time() - start_time
    print(f"Transformación completada en {transformation_time:.2f} segundos")
    
    return {
        "extraction_time": extraction_time,
        "transformation_time": transformation_time,
        "total_time": extraction_time + transformation_time,
        "row_count": len(df_pandas)
    }

def etl_with_polars():
    import polars as pl
    
    # Extracción
    start_time = time.time()
    print("Extrayendo datos con Polars...")
    df_polars = pl.read_parquet(TAXI_DATA_FILE)
    extraction_time = time.time() - start_time
    print(f"Extracción completada en {extraction_time:.2f} segundos")
    
    # Transformación
    start_time = time.time()
    print("Transformando datos con Polars...")
    
    # Renombrar columnas para consistencia
    column_mapping = {
        "VendorID": "vendor_id",
        "tpep_pickup_datetime": "pickup_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "PULocationID": "pickup_location_id",
        "DOLocationID": "dropoff_location_id"
    }
    for old_name, new_name in column_mapping.items():
        if old_name in df_polars.columns:
            df_polars = df_polars.rename({old_name: new_name})
    
    # Filtrar viajes con distancia válida
    df_polars = df_polars.filter(pl.col("trip_distance") > 0)
    
    # Filtrar viajes con tarifa válida
    df_polars = df_polars.filter(pl.col("fare_amount") >= 0)
    
    # Calcular la duración del viaje en minutos
    df_polars = df_polars.with_columns([
        ((pl.col("dropoff_datetime").dt.epoch() - pl.col("pickup_datetime").dt.epoch()) / 60).alias("trip_duration_minutes")
    ])
    
    # Filtrar viajes con duración válida
    df_polars = df_polars.filter(pl.col("trip_duration_minutes") > 0)
    
    # Calcular la velocidad promedio
    df_polars = df_polars.with_columns([
        (pl.col("trip_distance") / (pl.col("trip_duration_minutes") / 60)).alias("avg_speed_mph")
    ])
    
    # Filtrar velocidades razonables
    df_polars = df_polars.filter(pl.col("avg_speed_mph") < 100)
    
    # Manejar valores nulos
    df_polars = df_polars.with_columns([
        pl.col("passenger_count").fill_null(1),
        pl.col("congestion_surcharge").fill_null(0),
        pl.col("Airport_fee").fill_null(0)
    ])
    
    transformation_time = time.time() - start_time
    print(f"Transformación completada en {transformation_time:.2f} segundos")
    
    return {
        "extraction_time": extraction_time,
        "transformation_time": transformation_time,
        "total_time": extraction_time + transformation_time,
        "row_count": df_polars.shape[0]
    }

# Ejecutar ambas versiones y comparar
print("=== Benchmark: Pandas vs Polars ===")
print("\n1. Ejecutando ETL con Pandas...")
pandas_results = etl_with_pandas()

print("\n2. Ejecutando ETL con Polars...")
polars_results = etl_with_polars()

# Calcular la mejora de rendimiento
speedup_extraction = pandas_results["extraction_time"] / polars_results["extraction_time"]
speedup_transformation = pandas_results["transformation_time"] / polars_results["transformation_time"]
speedup_total = pandas_results["total_time"] / polars_results["total_time"]

print("\n=== Resultados del Benchmark ===")
print(f"Filas procesadas: {pandas_results['row_count']}")
print("\nTiempos de Pandas:")
print(f"  - Extracción: {pandas_results['extraction_time']:.2f} segundos")
print(f"  - Transformación: {pandas_results['transformation_time']:.2f} segundos")
print(f"  - Total: {pandas_results['total_time']:.2f} segundos")

print("\nTiempos de Polars:")
print(f"  - Extracción: {polars_results['extraction_time']:.2f} segundos")
print(f"  - Transformación: {polars_results['transformation_time']:.2f} segundos")
print(f"  - Total: {polars_results['total_time']:.2f} segundos")

print("\nMejora de rendimiento (Polars vs Pandas):")
print(f"  - Extracción: {speedup_extraction:.2f}x más rápido")
print(f"  - Transformación: {speedup_transformation:.2f}x más rápido")
print(f"  - Total: {speedup_total:.2f}x más rápido")

=== Benchmark: Pandas vs Polars ===

1. Ejecutando ETL con Pandas...
Extrayendo datos con Pandas...
Extracción completada en 4.52 segundos
Transformando datos con Pandas...
Transformación completada en 17.54 segundos

2. Ejecutando ETL con Polars...
Extrayendo datos con Polars...
Extracción completada en 1.59 segundos
Transformando datos con Polars...
Transformación completada en 2.74 segundos

=== Resultados del Benchmark ===
Filas procesadas: 39703334

Tiempos de Pandas:
  - Extracción: 4.52 segundos
  - Transformación: 17.54 segundos
  - Total: 22.07 segundos

Tiempos de Polars:
  - Extracción: 1.59 segundos
  - Transformación: 2.74 segundos
  - Total: 4.33 segundos

Mejora de rendimiento (Polars vs Pandas):
  - Extracción: 2.85x más rápido
  - Transformación: 6.40x más rápido
  - Total: 5.10x más rápido


## Ventajas de Polars para ETL

Basándonos en la implementación y los resultados del benchmark, podemos destacar las siguientes ventajas de Polars para procesos ETL:

1. **Rendimiento superior**: Como hemos visto en el benchmark, Polars es significativamente más rápido que Pandas en operaciones de extracción y transformación.

2. **Ejecución perezosa (lazy)**: Polars permite definir un plan de ejecución completo antes de ejecutarlo, lo que permite optimizaciones globales.

3. **Paralelismo automático**: Polars aprovecha automáticamente todos los núcleos disponibles sin configuración adicional.

4. **Eficiencia de memoria**: Polars consume menos memoria que Pandas para las mismas operaciones.

5. **API expresiva**: La API de Polars permite expresar transformaciones complejas de manera concisa y legible.

6. **Integración con ecosistema de datos**: Polars se integra bien con formatos como Parquet, CSV, JSON, etc.

7. **Consistencia de API**: La API de Polars es más consistente y predecible que la de Pandas.

Estas ventajas hacen de Polars una excelente opción para procesos ETL que manejan conjuntos de datos medianos a grandes en una sola máquina.

## Ventajas de la Arquitectura ETL Implementada

Nuestra arquitectura ETL combina varias tecnologías modernas para crear un flujo de trabajo robusto y eficiente:

1. **Polars para procesamiento de datos**: Aprovechamos el rendimiento y la expresividad de Polars para las operaciones de extracción y transformación.

2. **Pydantic para validación de datos**: Utilizamos Pydantic para asegurar que los datos cumplen con nuestras expectativas antes de cargarlos en la base de datos.

3. **SQLAlchemy para acceso a base de datos**: Utilizamos SQLAlchemy para definir el esquema de la base de datos y gestionar las conexiones de manera orientada a objetos.

4. **Logging para seguimiento**: Configuramos un sistema de logging para seguir el progreso del ETL y diagnosticar problemas.

Esta arquitectura proporciona:

- **Modularidad**: Cada componente tiene una responsabilidad clara y puede ser modificado o reemplazado independientemente.
- **Escalabilidad**: El diseño permite escalar a conjuntos de datos más grandes y flujos de trabajo más complejos.
- **Mantenibilidad**: El código está organizado de manera lógica y sigue buenas prácticas de ingeniería de software.
- **Robustez**: La validación de datos y el manejo de errores aseguran que el ETL sea resistente a problemas.
- **Observabilidad**: El logging y la monitorización permiten seguir el progreso y diagnosticar problemas.

## Conclusiones

En este ejemplo, hemos implementado un ETL completo utilizando Polars para procesar el dataset de taxis de Nueva York. Hemos demostrado las ventajas de Polars sobre Pandas en términos de rendimiento y funcionalidades, y hemos construido una arquitectura ETL robusta y eficiente.

Las principales conclusiones son:

1. **Polars ofrece un rendimiento significativamente mejor que Pandas** para operaciones ETL, especialmente en conjuntos de datos medianos a grandes.

2. **La combinación de Polars, Pydantic, SQLAlchemy y un Orquestador** proporciona una arquitectura ETL robusta, eficiente y mantenible.

3. **La validación estricta de tipos con Pydantic** asegura la integridad de los datos antes de cargarlos en la base de datos.

4. **La implementación de DAGs con Orquestador** permite definir flujos de trabajo complejos de manera clara y gestionar errores de manera efectiva.

5. **El logging y la monitorización** son esenciales para seguir el progreso del ETL y diagnosticar problemas.

En la siguiente sección, presentaremos un ejercicio práctico para que los estudiantes implementen su propio ETL utilizando estas tecnologías.